# Chapter 8: Your First LLM Integration
**From: Zero to AI Agent**

## Overview
In this chapter, you'll learn about:
- Setting up OpenAI API (or alternative)
- Making your first API call
- Understanding API responses
- Handling API errors gracefully
- Building a simple chatbot
- Saving conversation history


In [ ]:
!pip install -q -r requirements.txt

from dotenv import load_dotenv
load_dotenv()

---
## Section 8.1: Setting up OpenAI API (or alternative)

In [ ]:
# From: setup_api_key.py

# From: Zero to AI Agent, Chapter 8, Section 8.1
# File: setup_api_key.py

import os
from pathlib import Path

def setup_api_key():
    """Set up your OpenAI API key safely"""
    
    print("🔐 Let's set up your OpenAI API key!")
    print("=" * 50)
    
    # Check if we already have a key saved
    env_file = Path(".env")
    if env_file.exists():
        print("✅ You already have a .env file!")
        with open(env_file, 'r') as f:
            if 'OPENAI_API_KEY' in f.read():
                print("✅ Your API key is already set up!")
                return
    
    # Get the API key from user
    print("\nYou need your OpenAI API key from: https://platform.openai.com/api-keys")
    print("(It starts with 'sk-')")
    print()
    
    api_key = input("Paste your API key here: ").strip()
    
    # Basic validation
    if not api_key.startswith('sk-'):
        print("⚠️  That doesn't look like an OpenAI key (should start with 'sk-')")
        print("But let's save it anyway - you can fix it later!")
    
    # Save to .env file
    with open('.env', 'w') as f:
        f.write(f"OPENAI_API_KEY={api_key}\n")
    
    print("\n✅ API key saved to .env file!")
    print("🔒 Remember: NEVER share this file or commit it to Git!")
    
    # Create .gitignore to keep it safe
    with open('.gitignore', 'w') as f:
        f.write(".env\n")
    
    print("✅ Created .gitignore to keep your key safe!")
    print("\n🎉 You're all set! Let's talk to AI!")

if __name__ == "__main__":
    setup_api_key()


In [ ]:
# From: my_first_ai_chat.py

# From: Zero to AI Agent, Chapter 8, Section 8.1
# File: my_first_ai_chat.py

import openai
import os
from pathlib import Path

# Load your API key from .env file
def load_api_key():
    """Load API key from .env file"""
    env_file = Path(".env")
    if env_file.exists():
        with open(env_file, 'r') as f:
            for line in f:
                if line.startswith('OPENAI_API_KEY='):
                    return line.split('=')[1].strip()
    return None

# Get your API key
api_key = load_api_key()
if not api_key:
    print("❌ No API key found! Run setup_api_key.py first!")
    exit()

# THIS IS IT - YOUR FIRST AI CALL! 🚀
client = openai.OpenAI(api_key=api_key)

print("🤖 AI Assistant Ready!")
print("Type 'quit' to exit")
print("-" * 40)

# Conversation loop
while True:
    # Get your message
    user_message = input("\nYou: ")
    
    if user_message.lower() == 'quit':
        print("👋 Bye! You just built your first AI app!")
        break
    
    # ✨ THE MAGIC HAPPENS HERE - WE CALL THE AI! ✨
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": user_message}
        ]
    )
    
    # Get the AI's response
    ai_message = response.choices[0].message.content
    
    # Show the response
    print(f"\nAI: {ai_message}")

print("\n🎉 Congratulations! You just had your first AI conversation!")


In [ ]:
# From: ai_chat_with_memory.py

# From: Zero to AI Agent, Chapter 8, Section 8.1
# File: ai_chat_with_memory.py

import openai
import os
from pathlib import Path

# Load API key (same as before)
def load_api_key():
    env_file = Path(".env")
    if env_file.exists():
        with open(env_file, 'r') as f:
            for line in f:
                if line.startswith('OPENAI_API_KEY='):
                    return line.split('=')[1].strip()
    return None

# Setup
api_key = load_api_key()
if not api_key:
    print("❌ No API key found! Run setup_api_key.py first!")
    exit()

client = openai.OpenAI(api_key=api_key)

print("🤖 AI Assistant with Memory!")
print("I'll remember our conversation now!")
print("Commands: 'quit' to exit, 'forget' to clear memory")
print("-" * 40)

# This list will store our conversation history!
conversation = [
    {"role": "system", "content": "You are a helpful, friendly assistant."}
]

while True:
    user_message = input("\nYou: ")
    
    if user_message.lower() == 'quit':
        print("👋 Bye! Thanks for chatting!")
        break
    
    if user_message.lower() == 'forget':
        # Clear the conversation history
        conversation = [conversation[0]]  # Keep only system message
        print("🧹 Memory cleared! Fresh start!")
        continue
    
    # Add user message to conversation history
    conversation.append({"role": "user", "content": user_message})
    
    # Send the ENTIRE conversation to the AI
    # This is how it remembers what you talked about!
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=conversation  # <- The magic! Send all messages!
    )
    
    # Get and show the response
    ai_message = response.choices[0].message.content
    print(f"\nAI: {ai_message}")
    
    # Add AI's response to conversation history
    conversation.append({"role": "assistant", "content": ai_message})
    
    # Keep conversation from getting too long (optional)
    if len(conversation) > 20:
        # Keep system message and last 19 messages
        conversation = [conversation[0]] + conversation[-19:]

print("\n🎉 You just built a chatbot with memory!")


In [ ]:
# From: ai_chat_streaming.py

# From: Zero to AI Agent, Chapter 8, Section 8.1
# File: ai_chat_streaming.py

import openai
from pathlib import Path

# Load API key
def load_api_key():
    env_file = Path(".env")
    if env_file.exists():
        with open(env_file, 'r') as f:
            for line in f:
                if line.startswith('OPENAI_API_KEY='):
                    return line.split('=')[1].strip()
    return None

api_key = load_api_key()
client = openai.OpenAI(api_key=api_key)

print("🤖 Streaming AI Chat!")
print("Watch the AI 'type' its response!")
print("-" * 40)

while True:
    user_message = input("\nYou: ")
    
    if user_message.lower() == 'quit':
        break
    
    print("\nAI: ", end="", flush=True)
    
    # Add stream=True to see the response as it's generated!
    stream = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": user_message}],
        stream=True  # ← This makes it stream!
    )
    
    # Print each chunk as it arrives
    for chunk in stream:
        if chunk.choices[0].delta.content:
            print(chunk.choices[0].delta.content, end="", flush=True)
    
    print()  # New line after response

print("\n✨ Pretty cool, right?")


In [ ]:
# From: try_claude.py

# From: Zero to AI Agent, Chapter 8, Section 8.1
# File: try_claude.py (if you have an Anthropic API key)

import anthropic
from pathlib import Path

# Load API key from .env
def load_api_key():
    env_file = Path(".env")
    if env_file.exists():
        with open(env_file, 'r') as f:
            for line in f:
                if line.startswith('ANTHROPIC_API_KEY='):
                    return line.split('=')[1].strip()
    return None

api_key = load_api_key()
if not api_key:
    print("No Anthropic API key found. Add ANTHROPIC_API_KEY to your .env file!")
    exit()

# Create Claude client
client = anthropic.Anthropic(api_key=api_key)

# Ask Claude something!
message = client.messages.create(
    model="claude-3-sonnet-20240229",
    max_tokens=1000,
    messages=[
        {"role": "user", "content": "Tell me a short joke!"}
    ]
)

print("🤖 Claude says:")
print(message.content[0].text)


In [ ]:
# From: test_setup.py

# From: Zero to AI Agent, Chapter 8, Section 8.1
# File: test_setup.py

import sys
from pathlib import Path

print("🧪 Testing Your AI Setup")
print("=" * 40)

# Test 1: Check if OpenAI is installed
try:
    import openai
    print("✅ OpenAI package installed")
except ImportError:
    print("❌ OpenAI not installed. Run: pip install openai")
    sys.exit(1)

# Test 2: Check for API key
env_file = Path(".env")
if not env_file.exists():
    print("❌ No .env file found. Run: python setup_api_key.py")
    sys.exit(1)

api_key = None
with open(env_file, 'r') as f:
    for line in f:
        if line.startswith('OPENAI_API_KEY='):
            api_key = line.split('=')[1].strip()
            break

if not api_key or api_key == 'your-key-here':
    print("❌ No valid API key in .env file")
    sys.exit(1)

print("✅ API key found")

# Test 3: Try a real API call
print("🔄 Testing API connection...")
try:
    client = openai.OpenAI(api_key=api_key)
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": "Say 'test successful' in 3 words or less"}],
        max_tokens=10
    )
    result = response.choices[0].message.content
    print(f"✅ API call successful! AI said: {result}")
except Exception as e:
    print(f"❌ API call failed: {e}")
    sys.exit(1)

print("\n🎉 All tests passed! You're ready to build AI apps!")


---
### Section 8.1 Exercises

### Exercise 8.1.1: Personality Bot

Create a chatbot that:
- Has a specific personality (like a pirate, poet, or chef)
- Maintains that personality throughout the conversation
- Remembers previous messages
- Can switch personalities on command

 ** Solution**: `part_2_ai_basics/chapter_08_first_llm/exercise_1_8_1_solution.py`

In [ ]:
# Your code here


### Exercise 8.1.2: Conversation Saver

Build a chat program that:
- Saves each conversation to a text file
- Names the file with the current date and time
- Can load and continue a previous conversation
- Shows how many messages have been exchanged

 ** Solution**: `part_2_ai_basics/chapter_08_first_llm/exercise_2_8_1_solution.py`

In [ ]:
# Your code here


### Exercise 8.1.3: Simple API Cost Calculator

Create a tool that:
- Counts how many tokens you use in each message
- Estimates the cost of each API call
- Keeps a running total of your spending
- Warns you when you've spent more than $1

 ** Solution**: `part_2_ai_basics/chapter_08_first_llm/exercise_3_8_1_solution.py`

In [ ]:
# Your code here


---
## Section 8.2: Making your first API call

In [ ]:
# From: anatomy_of_api_call.py

# From: Zero to AI Agent, Chapter 8, Section 8.2
# File: anatomy_of_api_call.py

import openai
from pathlib import Path

# Load API key (you know this part!)
def load_api_key():
    env_file = Path(".env")
    if env_file.exists():
        with open(env_file, 'r') as f:
            for line in f:
                if line.startswith('OPENAI_API_KEY='):
                    return line.split('=')[1].strip()
    return None

api_key = load_api_key()
client = openai.OpenAI(api_key=api_key)

# Let's explore EVERY parameter you can use!
response = client.chat.completions.create(
    # 1. MODEL - Which AI brain to use
    model="gpt-3.5-turbo",  # Fast and cheap!
    # model="gpt-4",        # Smarter but slower
    # model="gpt-3.5-turbo-1106",  # Specific version
    
    # 2. MESSAGES - The conversation history
    messages=[
        # System message: Sets the AI's personality/role
        {"role": "system", "content": "You are a helpful assistant who explains things simply."},
        
        # User message: What the human says
        {"role": "user", "content": "What is Python?"},
        
        # Assistant message: Previous AI responses (for context)
        # {"role": "assistant", "content": "Previous response here..."},
    ],
    
    # 3. TEMPERATURE - Creativity control (0.0 to 2.0)
    temperature=0.7,  # 0 = focused/deterministic, 2 = very creative/random
    
    # 4. MAX_TOKENS - Maximum response length
    max_tokens=150,  # Roughly 1 token = 0.75 words
    
    # 5. TOP_P - Another way to control randomness (usually use temperature OR top_p, not both)
    top_p=1.0,  # 0.1 = only most likely words, 1.0 = consider all words
    
    # 6. FREQUENCY_PENALTY - Reduce repetition (-2.0 to 2.0)
    frequency_penalty=0.0,  # Positive = less repetition
    
    # 7. PRESENCE_PENALTY - Encourage new topics (-2.0 to 2.0)  
    presence_penalty=0.0,  # Positive = talk about new things
    
    # 8. STOP - Stop sequences (where to cut off response)
    stop=None,  # Can be a list like ["\n", ".", "END"]
    
    # 9. N - How many responses to generate
    n=1,  # Generate multiple responses and pick the best!
    
    # 10. STREAM - Get response as it's generated
    stream=False,  # True = see response word by word
    
    # 11. USER - Unique identifier for the user (for OpenAI's monitoring)
    user=None,  # Can be a user ID for tracking
)

# Understanding the response structure
print("🔍 API Response Structure:")
print(f"ID: {response.id}")
print(f"Model used: {response.model}")
print(f"Created at: {response.created}")

# The actual message
message = response.choices[0].message
print(f"\n💬 Response: {message.content}")

# Token usage (this is what costs money!)
if response.usage:
    print(f"\n📊 Token Usage:")
    print(f"  Prompt tokens: {response.usage.prompt_tokens}")
    print(f"  Response tokens: {response.usage.completion_tokens}")
    print(f"  Total tokens: {response.usage.total_tokens}")
    
    # Cost calculation (GPT-3.5-turbo pricing)
    cost = (response.usage.total_tokens / 1000) * 0.002
    print(f"  Estimated cost: ${cost:.6f}")

# Why the response was cut off (if applicable)
print(f"\n🛑 Finish reason: {response.choices[0].finish_reason}")
# Possible values:
# - "stop": Natural ending
# - "length": Hit max_tokens limit  
# - "content_filter": Blocked by safety filter
# - "null": Still generating (if streaming)


In [ ]:
# From: temperature_experiments.py

# From: Zero to AI Agent, Chapter 8, Section 8.2
# File: temperature_experiments.py

import openai
from pathlib import Path

def load_api_key():
    env_file = Path(".env")
    if env_file.exists():
        with open(env_file, 'r') as f:
            for line in f:
                if line.startswith('OPENAI_API_KEY='):
                    return line.split('=')[1].strip()
    return None

api_key = load_api_key()
client = openai.OpenAI(api_key=api_key)

def test_temperature(prompt, temperature, description):
    """Test how temperature affects responses"""
    print(f"\n🌡️ Temperature {temperature} - {description}")
    print("-" * 50)
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature,
        max_tokens=100
    )
    
    print(f"Response: {response.choices[0].message.content}")
    return response.choices[0].message.content

# Same prompt, different temperatures
prompt = "Write a one-sentence story about a robot"

print("🧪 Temperature Experiment: Same prompt, different creativity levels")
print("=" * 60)

# Temperature 0: Maximum consistency (almost deterministic)
test_temperature(prompt, 0, "Focused/Factual")

# Temperature 0.3: Slightly creative but still focused
test_temperature(prompt, 0.3, "Balanced/Professional")

# Temperature 0.7: Default - good balance
test_temperature(prompt, 0.7, "Creative/Natural")

# Temperature 1.0: More creative and varied
test_temperature(prompt, 1.0, "Very Creative")

# Temperature 1.5: Wild and unpredictable
test_temperature(prompt, 1.5, "Experimental/Wild")

print("\n" + "=" * 60)
print("💡 When to use different temperatures:")
print("  📊 0.0-0.3: Code generation, facts, math, analysis")
print("  📝 0.4-0.7: General chat, explanations, summaries")
print("  🎨 0.8-1.2: Creative writing, brainstorming, stories")
print("  🎲 1.3-2.0: Experimental, poetry, wild ideas")


In [ ]:
# From: simple_personalities.py

# From: Zero to AI Agent, Chapter 8, Section 8.2
# File: simple_personalities.py

import openai
from pathlib import Path

def load_api_key():
    env_file = Path(".env")
    if env_file.exists():
        with open(env_file, 'r') as f:
            for line in f:
                if line.startswith('OPENAI_API_KEY='):
                    return line.split('=')[1].strip()
    return None

api_key = load_api_key()
client = openai.OpenAI(api_key=api_key)

# Try different personalities with the SAME question
question = "How do I make friends?"

print("🎭 Same Question, 3 Different Personalities")
print("=" * 60)

# Personality 1: Friendly buddy
print("\n1️⃣ FRIENDLY BUDDY:")
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a friendly, casual buddy. Use simple language and be encouraging!"},
        {"role": "user", "content": question}
    ]
)
print(response.choices[0].message.content)

# Personality 2: Professional coach
print("\n2️⃣ PROFESSIONAL COACH:")
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a professional life coach. Be formal and give structured advice."},
        {"role": "user", "content": question}
    ]
)
print(response.choices[0].message.content)

# Personality 3: Wise grandparent
print("\n3️⃣ WISE GRANDPARENT:")
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a wise, caring grandparent. Share life wisdom and be warm."},
        {"role": "user", "content": question}
    ]
)
print(response.choices[0].message.content)

print("\n" + "=" * 60)
print("💡 See how the system message changes the AI's style?")
print("   You just learned how ChatGPT's 'Custom Instructions' work!")


In [ ]:
# From: smart_conversation.py

# From: Zero to AI Agent, Chapter 8, Section 8.2
# File: smart_conversation.py

import openai
from pathlib import Path

def load_api_key():
    env_file = Path(".env")
    if env_file.exists():
        with open(env_file, 'r') as f:
            for line in f:
                if line.startswith('OPENAI_API_KEY='):
                    return line.split('=')[1].strip()
    return None

api_key = load_api_key()
client = openai.OpenAI(api_key=api_key)

print("💬 Smart Conversation Manager")
print("Keeps only recent messages to save tokens!")
print("Commands: 'quit', 'count'")
print("-" * 40)

# Our conversation history
conversation = [
    {"role": "system", "content": "You are a helpful assistant."}
]

MAX_MESSAGES = 10  # Keep only last 10 messages

while True:
    user_input = input("\nYou: ")
    
    if user_input.lower() == 'quit':
        break
    
    if user_input.lower() == 'count':
        print(f"📊 Messages in memory: {len(conversation) - 1}")  # -1 for system message
        continue
    
    # Add user message
    conversation.append({"role": "user", "content": user_input})
    
    # Keep conversation from getting too long
    if len(conversation) > MAX_MESSAGES:
        # Keep system message (first) and recent messages
        conversation = [conversation[0]] + conversation[-(MAX_MESSAGES-1):]
        print("(Trimmed old messages to save tokens)")
    
    # Make API call with conversation history
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=conversation
    )
    
    ai_message = response.choices[0].message.content
    print(f"\nAI: {ai_message}")
    
    # Add AI response to history
    conversation.append({"role": "assistant", "content": ai_message})

print("\n✅ Smart conversation management - you're saving money already!")


In [ ]:
# From: streaming_demo.py

# From: Zero to AI Agent, Chapter 8, Section 8.2
# File: streaming_demo.py

import openai
from pathlib import Path

def load_api_key():
    env_file = Path(".env")
    if env_file.exists():
        with open(env_file, 'r') as f:
            for line in f:
                if line.startswith('OPENAI_API_KEY='):
                    return line.split('=')[1].strip()
    return None

api_key = load_api_key()
client = openai.OpenAI(api_key=api_key)

print("🌊 Streaming Chat Demo")
print("Watch the AI type its response!")
print("Type 'quit' to exit")
print("-" * 40)

while True:
    user_input = input("\nYou: ")
    
    if user_input.lower() == 'quit':
        break
    
    print("\nAI: ", end="", flush=True)
    
    # The magic parameter: stream=True
    stream = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": user_input}
        ],
        stream=True  # ← This makes it stream!
    )
    
    # Print each piece as it arrives
    for chunk in stream:
        if chunk.choices[0].delta.content:
            print(chunk.choices[0].delta.content, end="", flush=True)
    
    print()  # New line after response is complete

print("\n✨ That's how ChatGPT does it!")


In [ ]:
# From: my_complete_chatbot.py

# From: Zero to AI Agent, Chapter 8, Section 8.2
# File: my_complete_chatbot.py

import openai
from pathlib import Path
import json
from datetime import datetime

def load_api_key():
    env_file = Path(".env")
    if env_file.exists():
        with open(env_file, 'r') as f:
            for line in f:
                if line.startswith('OPENAI_API_KEY='):
                    return line.split('=')[1].strip()
    return None

api_key = load_api_key()
if not api_key:
    print("❌ No API key found! Run setup_api_key.py first!")
    exit()

client = openai.OpenAI(api_key=api_key)

print("🤖 Your Complete Chatbot")
print("=" * 50)
print("Features:")
print("  • Adjustable temperature (creative vs focused)")
print("  • Streaming responses")
print("  • Conversation memory")
print("  • Save conversations")
print("\nCommands: 'quit', 'save', 'temp <value>', 'clear'")
print("=" * 50)

# Settings
temperature = 0.7
conversation = [{"role": "system", "content": "You are a helpful, friendly assistant."}]

while True:
    user_input = input("\nYou: ").strip()
    
    # Handle commands
    if user_input.lower() == 'quit':
        print("👋 Goodbye!")
        break
    
    elif user_input.lower() == 'clear':
        conversation = [conversation[0]]  # Keep only system message
        print("🧹 Conversation cleared!")
        continue
    
    elif user_input.lower() == 'save':
        # Save conversation to file
        filename = f"chat_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        with open(filename, 'w') as f:
            json.dump({
                "conversation": conversation,
                "temperature": temperature,
                "timestamp": datetime.now().isoformat()
            }, f, indent=2)
        print(f"💾 Saved to {filename}")
        continue
    
    elif user_input.lower().startswith('temp '):
        # Change temperature
        try:
            new_temp = float(user_input.split()[1])
            if 0 <= new_temp <= 2:
                temperature = new_temp
                print(f"🌡️ Temperature set to {temperature}")
            else:
                print("Temperature must be between 0 and 2")
        except:
            print("Invalid temperature")
        continue
    
    # Regular chat message
    if user_input:
        # Add to conversation
        conversation.append({"role": "user", "content": user_input})
        
        # Keep conversation manageable (max 20 messages)
        if len(conversation) > 20:
            conversation = [conversation[0]] + conversation[-19:]
        
        print("\nAI: ", end="", flush=True)
        
        try:
            # Stream the response
            stream = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=conversation,
                temperature=temperature,
                stream=True
            )
            
            full_response = ""
            for chunk in stream:
                if chunk.choices[0].delta.content:
                    content = chunk.choices[0].delta.content
                    print(content, end="", flush=True)
                    full_response += content
            
            print()  # New line
            
            # Add AI response to conversation
            conversation.append({"role": "assistant", "content": full_response})
            
        except Exception as e:
            print(f"\n❌ Error: {e}")

print("\n🎉 Thanks for using your complete chatbot!")


In [ ]:
# From: mini_chatgpt.py

# From: Zero to AI Agent, Chapter 8, Section 8.2
# File: mini_chatgpt.py

import openai
from pathlib import Path
import json
from datetime import datetime
import sys

def load_api_key():
    env_file = Path(".env")
    if env_file.exists():
        with open(env_file, 'r') as f:
            for line in f:
                if line.startswith('OPENAI_API_KEY='):
                    return line.split('=')[1].strip()
    return None

class MiniChatGPT:
    """Your own mini version of ChatGPT!"""
    
    def __init__(self, api_key):
        self.client = openai.OpenAI(api_key=api_key)
        self.conversations = {}  # Multiple conversation threads
        self.current_conversation_id = None
        self.settings = {
            "model": "gpt-3.5-turbo",
            "temperature": 0.7,
            "max_tokens": 500,
            "stream": True
        }
    
    def new_conversation(self, title=None):
        """Start a new conversation thread"""
        conv_id = datetime.now().strftime("%Y%m%d_%H%M%S")
        title = title or f"Chat {conv_id}"
        
        self.conversations[conv_id] = {
            "id": conv_id,
            "title": title,
            "messages": [],
            "created": datetime.now().isoformat(),
            "token_count": 0,
            "cost": 0.0
        }
        
        self.current_conversation_id = conv_id
        return conv_id
    
    def list_conversations(self):
        """List all conversations"""
        if not self.conversations:
            print("No conversations yet.")
            return
        
        print("\n📚 Your Conversations:")
        for i, (conv_id, conv) in enumerate(self.conversations.items(), 1):
            msg_count = len(conv["messages"])
            print(f"  {i}. {conv['title']} ({msg_count} messages)")
            if conv_id == self.current_conversation_id:
                print("     ^ Current")
    
    def switch_conversation(self, conv_id):
        """Switch to a different conversation"""
        if conv_id in self.conversations:
            self.current_conversation_id = conv_id
            print(f"Switched to: {self.conversations[conv_id]['title']}")
            return True
        return False
    
    def get_current_messages(self):
        """Get messages for current conversation"""
        if not self.current_conversation_id:
            self.new_conversation()
        
        conv = self.conversations[self.current_conversation_id]
        
        # System message + conversation history
        messages = [
            {"role": "system", "content": "You are ChatGPT, a helpful AI assistant."}
        ]
        
        # Add conversation messages (limit to last 20 for token management)
        for msg in conv["messages"][-20:]:
            messages.append({
                "role": msg["role"],
                "content": msg["content"]
            })
        
        return messages
    
    def chat(self, user_message):
        """Send message and get response"""
        if not self.current_conversation_id:
            self.new_conversation()
        
        conv = self.conversations[self.current_conversation_id]
        
        # Add user message
        conv["messages"].append({
            "role": "user",
            "content": user_message,
            "timestamp": datetime.now().isoformat()
        })
        
        # Get messages for API
        messages = self.get_current_messages()
        
        print("\n🤖 ChatGPT: ", end="", flush=True)
        
        full_response = ""
        
        try:
            if self.settings["stream"]:
                # Streaming response
                stream = self.client.chat.completions.create(
                    model=self.settings["model"],
                    messages=messages,
                    temperature=self.settings["temperature"],
                    max_tokens=self.settings["max_tokens"],
                    stream=True
                )
                
                for chunk in stream:
                    if chunk.choices[0].delta.content:
                        content = chunk.choices[0].delta.content
                        print(content, end="", flush=True)
                        full_response += content
                
            else:
                # Non-streaming response
                response = self.client.chat.completions.create(
                    model=self.settings["model"],
                    messages=messages,
                    temperature=self.settings["temperature"],
                    max_tokens=self.settings["max_tokens"]
                )
                
                full_response = response.choices[0].message.content
                print(full_response)
                
                # Track usage
                if response.usage:
                    conv["token_count"] += response.usage.total_tokens
                    conv["cost"] += (response.usage.total_tokens / 1000) * 0.002
            
            print()  # New line
            
            # Add assistant response
            conv["messages"].append({
                "role": "assistant",
                "content": full_response,
                "timestamp": datetime.now().isoformat()
            })
            
            return full_response
            
        except Exception as e:
            print(f"\n❌ Error: {e}")
            return None
    
    def save_all_conversations(self):
        """Save all conversations to file"""
        filename = f"conversations_backup_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        
        with open(filename, 'w') as f:
            json.dump(self.conversations, f, indent=2)
        
        print(f"💾 Saved all conversations to {filename}")
    
    def show_settings(self):
        """Show current settings"""
        print("\n⚙️ Current Settings:")
        for key, value in self.settings.items():
            print(f"  {key}: {value}")
    
    def change_setting(self, key, value):
        """Change a setting"""
        if key in self.settings:
            old_value = self.settings[key]
            self.settings[key] = value
            print(f"✅ Changed {key}: {old_value} → {value}")
        else:
            print(f"❌ Unknown setting: {key}")
    
    def show_help(self):
        """Show available commands"""
        print("\n📖 Available Commands:")
        print("  'new' - Start new conversation")
        print("  'list' - List all conversations")
        print("  'switch' - Switch to another conversation")
        print("  'settings' - Show current settings")
        print("  'temp <value>' - Change temperature (0-2)")
        print("  'model <name>' - Change model")
        print("  'save' - Save all conversations")
        print("  'help' - Show this help")
        print("  'quit' - Exit")
    
    def run(self):
        """Run the ChatGPT clone"""
        print("🚀 Mini ChatGPT")
        print("=" * 60)
        print("Welcome to your own ChatGPT clone!")
        print("Type 'help' for commands or just start chatting!")
        print("=" * 60)
        
        # Start first conversation
        self.new_conversation("Welcome Chat")
        
        while True:
            try:
                user_input = input("\n👤 You: ").strip()
                
                if not user_input:
                    continue
                
                # Check for commands
                if user_input.lower() == 'quit':
                    print("\n👋 Goodbye!")
                    break
                
                elif user_input.lower() == 'help':
                    self.show_help()
                
                elif user_input.lower() == 'new':
                    title = input("Conversation title (or Enter for default): ").strip()
                    conv_id = self.new_conversation(title)
                    print(f"✨ Started new conversation: {self.conversations[conv_id]['title']}")
                
                elif user_input.lower() == 'list':
                    self.list_conversations()
                
                elif user_input.lower() == 'switch':
                    self.list_conversations()
                    choice = input("Choose conversation number: ").strip()
                    try:
                        idx = int(choice) - 1
                        conv_list = list(self.conversations.keys())
                        if 0 <= idx < len(conv_list):
                            self.switch_conversation(conv_list[idx])
                    except:
                        print("Invalid choice")
                
                elif user_input.lower() == 'settings':
                    self.show_settings()
                
                elif user_input.lower().startswith('temp '):
                    try:
                        temp = float(user_input.split()[1])
                        if 0 <= temp <= 2:
                            self.change_setting('temperature', temp)
                        else:
                            print("Temperature must be between 0 and 2")
                    except:
                        print("Invalid temperature value")
                
                elif user_input.lower().startswith('model '):
                    model = user_input[6:].strip()
                    self.change_setting('model', model)
                
                elif user_input.lower() == 'save':
                    self.save_all_conversations()
                
                else:
                    # Regular chat message
                    self.chat(user_input)
                    
            except KeyboardInterrupt:
                print("\n⚠️ Interrupted! Type 'quit' to exit.")
            except Exception as e:
                print(f"\n❌ Error: {e}")

# Run Mini ChatGPT!
if __name__ == "__main__":
    api_key = load_api_key()
    if not api_key:
        print("❌ No API key found! Run setup_api_key.py first!")
        exit()
    
    chatgpt = MiniChatGPT(api_key)
    chatgpt.run()


---
### Section 8.2 Exercises

### Exercise 8.2.1: Temperature Tester

Create a simple tool that:
- Takes one prompt from the user
- Generates 3 responses at different temperatures (0, 0.7, 1.5)
- Shows all three responses
- Lets the user pick their favorite

 ** Solution**: `part_2_ai_basics/chapter_08_first_llm/exercise_1_8_2_solution.py`

In [ ]:
# Your code here


### Exercise 8.2.2: Conversation Counter

Build a chatbot that:
- Counts how many messages have been sent
- Shows the word count of each response
- Estimates the cost (roughly $0.002 per 1000 tokens)
- Saves the conversation when you quit

 ** Solution**: `part_2_ai_basics/chapter_08_first_llm/exercise_2_8_2_solution.py`

In [ ]:
# Your code here


### Exercise 8.2.3: Personality Switcher

Create a chatbot that:
- Has 3 different personalities (your choice!)
- Lets you switch between them with a command
- Each personality has a different temperature setting
- Shows which personality is currently active

 ** Solution**: `part_2_ai_basics/chapter_08_first_llm/exercise_3_8_2_solution.py`

 **BONUS CHALLENGE: Complete the Mini ChatGPT!**
Take the MiniChatGPT class boilerplate from above and implement all the methods to create a full ChatGPT clone with:
- Multiple conversation threads (like ChatGPT's sidebar!)
- Ability to switch between conversations
- List all your conversations
- Save everything to a file
- Each conversation remembers its own history

This is the ultimate test of everything you've learned - can you build your own ChatGPT?

In [ ]:
# Your code here


---
## Section 8.3: Understanding API responses

In [ ]:
# From: response_inspector.py

# From: Zero to AI Agent, Chapter 8, Section 8.3
# File: response_inspector.py

# Simple tools to understand API responses

def inspect_response(response):
    """Print all the interesting parts of a response"""
    print("\n🔍 Response Inspector")
    print("=" * 50)
    
    # Basic info
    print(f"Response ID: {response.id}")
    print(f"Model: {response.model}")
    print(f"Created: {response.created}")
    
    # The actual message
    if response.choices:
        message = response.choices[0].message.content
        print(f"\nMessage: {message[:100]}...")
        print(f"Finish reason: {response.choices[0].finish_reason}")
    
    # Token usage
    if hasattr(response, 'usage') and response.usage:
        print(f"\n📊 Token Usage:")
        print(f"  Prompt: {response.usage.prompt_tokens}")
        print(f"  Response: {response.usage.completion_tokens}")
        print(f"  Total: {response.usage.total_tokens}")
    
    print("=" * 50)

def calculate_simple_cost(tokens, model="gpt-3.5-turbo"):
    """Simple cost calculator"""
    # Rough pricing (as of 2024)
    if model == "gpt-3.5-turbo":
        return tokens * 0.002 / 1000  # $0.002 per 1K tokens
    elif model == "gpt-4":
        return tokens * 0.03 / 1000   # $0.03 per 1K tokens
    return 0


In [ ]:
# From: explore_responses.py

# From: Zero to AI Agent, Chapter 8, Section 8.3
# File: explore_responses.py

import openai
from pathlib import Path
from response_inspector import inspect_response, calculate_simple_cost

def load_api_key():
    env_file = Path(".env")
    if env_file.exists():
        with open(env_file, 'r') as f:
            for line in f:
                if line.startswith('OPENAI_API_KEY='):
                    return line.split('=')[1].strip()
    return None

# Setup
api_key = load_api_key()
if not api_key:
    print("❌ No API key found!")
    exit()

client = openai.OpenAI(api_key=api_key)

# Try a simple request
print("📤 Sending request: 'Say hello in 5 words'")
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Say hello in 5 words"}],
    temperature=0.7
)

# Inspect what came back
inspect_response(response)

# Calculate cost
if response.usage:
    cost = calculate_simple_cost(response.usage.total_tokens)
    print(f"\n💰 Estimated cost: ${cost:.6f}")

# What's in the response object?
print("\n📦 Response object attributes:")
for attr in dir(response):
    if not attr.startswith('_'):
        print(f"  • {attr}")


In [ ]:
# From: token_explorer.py

# From: Zero to AI Agent, Chapter 8, Section 8.3
# File: token_explorer.py

import openai
from pathlib import Path

def load_api_key():
    env_file = Path(".env")
    if env_file.exists():
        with open(env_file, 'r') as f:
            for line in f:
                if line.startswith('OPENAI_API_KEY='):
                    return line.split('=')[1].strip()
    return None

def estimate_tokens(text):
    """Rough estimate: 1 token ≈ 4 characters or 0.75 words"""
    return max(len(text) // 4, len(text.split()) * 4 // 3)

# Setup
api_key = load_api_key()
client = openai.OpenAI(api_key=api_key)

# Test different prompts
test_prompts = [
    "Hi",
    "Hello, how are you today?",
    "Write a haiku about coding",
    "Explain quantum computing in simple terms",
]

print("🔬 Token Usage Explorer")
print("=" * 50)

for prompt in test_prompts:
    print(f"\n📝 Prompt: '{prompt}'")
    print(f"📏 Estimated tokens: {estimate_tokens(prompt)}")
    
    # Make API call
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    
    # Get actual token counts
    if response.usage:
        print(f"✅ Actual prompt tokens: {response.usage.prompt_tokens}")
        print(f"✅ Response tokens: {response.usage.completion_tokens}")
        print(f"✅ Total tokens: {response.usage.total_tokens}")
        
        # Cost
        cost = response.usage.total_tokens * 0.002 / 1000
        print(f"💰 Cost: ${cost:.6f}")
    
    # Show the response
    message = response.choices[0].message.content
    print(f"💬 Response: {message[:50]}...")

print("\n" + "=" * 50)
print("💡 Tip: Shorter prompts = lower costs!")


In [ ]:
# From: finish_reason_demo.py

# From: Zero to AI Agent, Chapter 8, Section 8.3
# File: finish_reason_demo.py

import openai
from pathlib import Path

def load_api_key():
    env_file = Path(".env")
    if env_file.exists():
        with open(env_file, 'r') as f:
            for line in f:
                if line.startswith('OPENAI_API_KEY='):
                    return line.split('=')[1].strip()
    return None

def explain_finish_reason(reason):
    """Explain what each finish reason means"""
    reasons = {
        "stop": "✅ Natural ending - AI completed its thought",
        "length": "✂️ Hit token limit - response was cut off",
        "content_filter": "🚫 Blocked by safety filter",
        "null": "⏳ Still generating (in streaming)",
    }
    return reasons.get(reason, f"❓ Unknown: {reason}")

# Setup
api_key = load_api_key()
client = openai.OpenAI(api_key=api_key)

print("🔬 Finish Reason Explorer")
print("=" * 50)

# Test 1: Natural completion
print("\n1️⃣ Natural completion test:")
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Count to 5"}],
)
print(f"Response: {response.choices[0].message.content}")
print(f"Finish reason: {response.choices[0].finish_reason}")
print(explain_finish_reason(response.choices[0].finish_reason))

# Test 2: Length limit
print("\n2️⃣ Length limit test:")
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Tell me a long story"}],
    max_tokens=10  # Very short limit!
)
print(f"Response: {response.choices[0].message.content}")
print(f"Finish reason: {response.choices[0].finish_reason}")
print(explain_finish_reason(response.choices[0].finish_reason))

# Test 3: Quick task
print("\n3️⃣ Quick task test:")
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "What's 2+2?"}],
)
print(f"Response: {response.choices[0].message.content}")
print(f"Finish reason: {response.choices[0].finish_reason}")
print(explain_finish_reason(response.choices[0].finish_reason))

print("\n" + "=" * 50)
print("💡 Understanding finish reasons helps you:")
print("  • Know if responses are complete")
print("  • Detect when you need higher token limits")
print("  • Handle truncated responses properly")


In [ ]:
# From: response_comparison.py

# From: Zero to AI Agent, Chapter 8, Section 8.3
# File: response_comparison.py

import openai
from pathlib import Path

def load_api_key():
    env_file = Path(".env")
    if env_file.exists():
        with open(env_file, 'r') as f:
            for line in f:
                if line.startswith('OPENAI_API_KEY='):
                    return line.split('=')[1].strip()
    return None

# Setup
api_key = load_api_key()
client = openai.OpenAI(api_key=api_key)

prompt = "Write a creative tagline for a coffee shop"

print("☕ Comparing Different Temperatures")
print("=" * 50)
print(f"Prompt: {prompt}\n")

temperatures = [0.2, 0.7, 1.2]
total_tokens = 0
total_cost = 0

for temp in temperatures:
    print(f"\n🌡️ Temperature: {temp}")
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=temp
    )
    
    message = response.choices[0].message.content
    tokens = response.usage.total_tokens
    cost = tokens * 0.002 / 1000
    
    print(f"Response: {message}")
    print(f"Tokens: {tokens}")
    print(f"Cost: ${cost:.6f}")
    
    total_tokens += tokens
    total_cost += cost

print("\n" + "=" * 50)
print(f"📊 Totals:")
print(f"  Total tokens: {total_tokens}")
print(f"  Total cost: ${total_cost:.6f}")
print(f"  Average per response: ${total_cost/len(temperatures):.6f}")


In [ ]:
# From: simple_logger.py

# From: Zero to AI Agent, Chapter 8, Section 8.3
# File: simple_logger.py

import openai
import json
from pathlib import Path
from datetime import datetime

def load_api_key():
    env_file = Path(".env")
    if env_file.exists():
        with open(env_file, 'r') as f:
            for line in f:
                if line.startswith('OPENAI_API_KEY='):
                    return line.split('=')[1].strip()
    return None

def log_response(prompt, response):
    """Log response details to a file"""
    log_entry = {
        "timestamp": datetime.now().isoformat(),
        "prompt": prompt,
        "response": response.choices[0].message.content,
        "model": response.model,
        "tokens": {
            "prompt": response.usage.prompt_tokens,
            "completion": response.usage.completion_tokens,
            "total": response.usage.total_tokens
        },
        "finish_reason": response.choices[0].finish_reason,
        "cost_estimate": response.usage.total_tokens * 0.002 / 1000
    }
    
    # Append to log file
    log_file = "api_responses.json"
    
    # Load existing logs
    if Path(log_file).exists():
        with open(log_file, 'r') as f:
            logs = json.load(f)
    else:
        logs = []
    
    # Add new entry
    logs.append(log_entry)
    
    # Save
    with open(log_file, 'w') as f:
        json.dump(logs, f, indent=2)
    
    print(f"📝 Logged response to {log_file}")
    return log_entry

# Setup
api_key = load_api_key()
client = openai.OpenAI(api_key=api_key)

print("📊 Response Logger")
print("Type your prompts and I'll log all the details!")
print("Type 'quit' to exit, 'stats' to see statistics")
print("=" * 50)

while True:
    user_input = input("\nYour prompt: ").strip()
    
    if user_input.lower() == 'quit':
        break
    
    if user_input.lower() == 'stats':
        # Show statistics from log
        if Path("api_responses.json").exists():
            with open("api_responses.json", 'r') as f:
                logs = json.load(f)
            
            total_tokens = sum(log['tokens']['total'] for log in logs)
            total_cost = sum(log['cost_estimate'] for log in logs)
            
            print(f"\n📈 Statistics from {len(logs)} requests:")
            print(f"  Total tokens: {total_tokens:,}")
            print(f"  Total cost: ${total_cost:.6f}")
            print(f"  Average per request: ${total_cost/len(logs):.6f}")
        else:
            print("No logs yet!")
        continue
    
    # Make API call
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": user_input}]
    )
    
    # Show response
    print(f"\n🤖 Response: {response.choices[0].message.content}")
    
    # Log it
    log_entry = log_response(user_input, response)
    print(f"📊 Used {log_entry['tokens']['total']} tokens (${log_entry['cost_estimate']:.6f})")

print("\n👋 Check api_responses.json for your logged data!")


---
### Section 8.3 Exercises

### Exercise 8.3.1: Token Predictor

Create a tool that:
- Takes a prompt from the user
- Predicts how many tokens it will use
- Makes the API call
- Compares prediction vs actual
- Keeps score of your accuracy

 ** Solution**: `part_2_ai_basics/chapter_08_first_llm/exercise_1_8_3_solution.py`

In [ ]:
# Your code here


### Exercise 8.3.2: Response Time Tracker

Build a simple tool that:
- Measures how long API calls take
- Tests different prompt lengths
- Shows if longer prompts take more time
- Finds the sweet spot for speed vs detail

 ** Solution**: `part_2_ai_basics/chapter_08_first_llm/exercise_2_8_3_solution.py`

In [ ]:
# Your code here


### Exercise 8.3.3: Model Comparison

Create a comparison tool that:
- Sends the same prompt to different models (if available)
- Compares token usage
- Compares costs
- Shows the differences in responses

 ** Solution**: `part_2_ai_basics/chapter_08_first_llm/exercise_3_8_3_solution.py`

In [ ]:
# Your code here


---
## Section 8.4: Handling API errors gracefully

In [ ]:
# From: error_types_demo.py

# From: Zero to AI Agent, Chapter 8, Section 8.4
# File: error_types_demo.py

# Let's trigger different types of errors and see what happens!

import openai
from pathlib import Path
import time

def load_api_key():
    env_file = Path(".env")
    if env_file.exists():
        with open(env_file, 'r') as f:
            for line in f:
                if line.startswith('OPENAI_API_KEY='):
                    return line.split('=')[1].strip()
    return None

def test_error(description, test_function):
    """Test an error condition and show what happens"""
    print(f"\n🧪 Testing: {description}")
    print("-" * 40)
    try:
        test_function()
        print("✅ No error occurred")
    except Exception as e:
        error_type = type(e).__name__
        print(f"❌ Error Type: {error_type}")
        print(f"📝 Error Message: {str(e)}")
        
        # Show useful error details if available
        if hasattr(e, 'response'):
            print(f"📊 Status Code: {getattr(e, 'status_code', 'N/A')}")
        
        return error_type
    return None

# Setup
print("🔬 API Error Types Explorer")
print("=" * 50)

# Test 1: Invalid API Key
print("\n1️⃣ Invalid API Key Test:")
try:
    bad_client = openai.OpenAI(api_key="sk-invalid-key-12345")
    response = bad_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": "Hi"}]
    )
except Exception as e:
    print(f"❌ {type(e).__name__}: {str(e)[:100]}...")

# Test 2: No API Key
print("\n2️⃣ Missing API Key Test:")
try:
    no_key_client = openai.OpenAI(api_key=None)
    response = no_key_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": "Hi"}]
    )
except Exception as e:
    print(f"❌ {type(e).__name__}: {str(e)[:100]}...")

# Test 3: Invalid Model
api_key = load_api_key()
if api_key:
    client = openai.OpenAI(api_key=api_key)
    
    print("\n3️⃣ Invalid Model Test:")
    try:
        response = client.chat.completions.create(
            model="gpt-99-ultra",  # This model doesn't exist!
            messages=[{"role": "user", "content": "Hi"}]
        )
    except Exception as e:
        print(f"❌ {type(e).__name__}: {str(e)[:100]}...")
    
    # Test 4: Invalid Parameters
    print("\n4️⃣ Invalid Parameters Test:")
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": "Hi"}],
            temperature=5.0  # Too high! Max is 2.0
        )
    except Exception as e:
        print(f"❌ {type(e).__name__}: {str(e)[:100]}...")
    
    # Test 5: Empty Messages
    print("\n5️⃣ Empty Messages Test:")
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[]  # No messages!
        )
    except Exception as e:
        print(f"❌ {type(e).__name__}: {str(e)[:100]}...")

print("\n" + "=" * 50)
print("💡 Common error types:")
print("  • AuthenticationError: Bad API key")
print("  • NotFoundError: Invalid model or endpoint")
print("  • BadRequestError: Invalid parameters")
print("  • RateLimitError: Too many requests")
print("  • APIConnectionError: Network issues")


In [ ]:
# From: basic_error_handling.py

# From: Zero to AI Agent, Chapter 8, Section 8.4
# File: basic_error_handling.py

import openai
from pathlib import Path
import sys

def load_api_key():
    env_file = Path(".env")
    if env_file.exists():
        with open(env_file, 'r') as f:
            for line in f:
                if line.startswith('OPENAI_API_KEY='):
                    return line.split('=')[1].strip()
    return None

def safe_api_call(client, messages, max_tokens=None):
    """Make an API call with proper error handling"""
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            max_tokens=max_tokens
        )
        return response.choices[0].message.content, None
    
    except openai.AuthenticationError:
        return None, "❌ Invalid API key. Please check your credentials."
    
    except openai.RateLimitError as e:
        return None, "⏳ Rate limit hit. Please wait a moment and try again."
    
    except openai.BadRequestError as e:
        return None, f"❌ Invalid request: {str(e)}"
    
    except openai.APIConnectionError:
        return None, "🌐 Network error. Please check your internet connection."
    
    except openai.APITimeoutError:
        return None, "⏱️ Request timed out. Please try again."
    
    except Exception as e:
        # Catch any other errors
        return None, f"❌ Unexpected error: {type(e).__name__}: {str(e)}"

# Setup
api_key = load_api_key()
if not api_key:
    print("❌ No API key found! Please set up your .env file.")
    sys.exit(1)

client = openai.OpenAI(api_key=api_key)

print("🛡️ Safe API Caller")
print("=" * 50)
print("This handles errors gracefully!")
print("Type 'quit' to exit")
print("-" * 50)

while True:
    user_input = input("\nYour message: ").strip()
    
    if user_input.lower() == 'quit':
        break
    
    # Make safe API call
    messages = [{"role": "user", "content": user_input}]
    response, error = safe_api_call(client, messages)
    
    if error:
        print(f"\n{error}")
        print("💡 Tip: The application didn't crash! You can try again.")
    else:
        print(f"\n🤖 Response: {response}")

print("\n👋 Goodbye!")


In [ ]:
# From: smart_retry.py

# From: Zero to AI Agent, Chapter 8, Section 8.4
# File: smart_retry.py

import openai
from pathlib import Path
import time
import random

def load_api_key():
    env_file = Path(".env")
    if env_file.exists():
        with open(env_file, 'r') as f:
            for line in f:
                if line.startswith('OPENAI_API_KEY='):
                    return line.split('=')[1].strip()
    return None

def exponential_backoff(attempt):
    """Calculate wait time with exponential backoff"""
    # 1st attempt: 1 sec, 2nd: 2 sec, 3rd: 4 sec, etc.
    base_wait = 2 ** attempt
    
    # Add jitter to prevent thundering herd
    jitter = random.uniform(0, 0.5)
    
    return min(base_wait + jitter, 32)  # Max 32 seconds

def api_call_with_retry(client, messages, max_retries=3):
    """Make API call with automatic retry on failure"""
    last_error = None
    
    for attempt in range(max_retries):
        try:
            print(f"🔄 Attempt {attempt + 1}/{max_retries}...")
            
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=messages
            )
            
            print("✅ Success!")
            return response.choices[0].message.content, None
        
        except openai.RateLimitError as e:
            last_error = e
            if attempt < max_retries - 1:
                wait_time = exponential_backoff(attempt)
                print(f"⏳ Rate limit hit. Waiting {wait_time:.1f} seconds...")
                time.sleep(wait_time)
            else:
                print("❌ Rate limit persists after retries")
        
        except openai.APITimeoutError as e:
            last_error = e
            if attempt < max_retries - 1:
                wait_time = exponential_backoff(attempt)
                print(f"⏱️ Timeout. Retrying in {wait_time:.1f} seconds...")
                time.sleep(wait_time)
            else:
                print("❌ Timeout after all retries")
        
        except openai.APIConnectionError as e:
            last_error = e
            if attempt < max_retries - 1:
                wait_time = exponential_backoff(attempt)
                print(f"🌐 Connection error. Retrying in {wait_time:.1f} seconds...")
                time.sleep(wait_time)
            else:
                print("❌ Connection failed after all retries")
        
        except openai.AuthenticationError:
            # Don't retry auth errors - they won't fix themselves
            print("❌ Authentication failed - check your API key")
            return None, "Invalid API key"
        
        except Exception as e:
            # Don't retry unexpected errors
            print(f"❌ Unexpected error: {type(e).__name__}")
            return None, str(e)
    
    # All retries failed
    return None, f"Failed after {max_retries} attempts: {str(last_error)}"

# Test the retry mechanism
api_key = load_api_key()
if not api_key:
    print("❌ No API key found!")
    exit()

client = openai.OpenAI(api_key=api_key)

print("🔄 Smart Retry Demo")
print("=" * 50)
print("Testing retry mechanism with different scenarios")
print("-" * 50)

# Test 1: Normal request (should succeed on first try)
print("\n📝 Test 1: Normal request")
messages = [{"role": "user", "content": "Say hello!"}]
response, error = api_call_with_retry(client, messages)
if response:
    print(f"Response: {response}")

# Test 2: Simulate rate limit (you can trigger this with rapid requests)
print("\n📝 Test 2: Multiple rapid requests")
for i in range(3):
    print(f"\nRequest {i+1}:")
    messages = [{"role": "user", "content": f"Count to {i+1}"}]
    response, error = api_call_with_retry(client, messages)
    if response:
        print(f"Response: {response[:50]}...")
    else:
        print(f"Failed: {error}")

print("\n" + "=" * 50)
print("💡 Retry best practices:")
print("  • Use exponential backoff (wait longer each retry)")
print("  • Add jitter to prevent synchronized retries")
print("  • Don't retry authentication errors")
print("  • Set a reasonable max retry limit")


In [ ]:
# From: rate_limit_handler.py

# From: Zero to AI Agent, Chapter 8, Section 8.4
# File: rate_limit_handler.py

import openai
from pathlib import Path
import time
from datetime import datetime, timedelta

def load_api_key():
    env_file = Path(".env")
    if env_file.exists():
        with open(env_file, 'r') as f:
            for line in f:
                if line.startswith('OPENAI_API_KEY='):
                    return line.split('=')[1].strip()
    return None

class RateLimitHandler:
    """Handle rate limits intelligently"""
    
    def __init__(self, client):
        self.client = client
        self.request_times = []
        self.requests_per_minute = 20  # Conservative limit
        self.last_rate_limit_time = None
        
    def wait_if_needed(self):
        """Check if we need to wait before making a request"""
        now = datetime.now()
        
        # Clean old request times (older than 1 minute)
        one_minute_ago = now - timedelta(minutes=1)
        self.request_times = [t for t in self.request_times if t > one_minute_ago]
        
        # Check if we're approaching the limit
        if len(self.request_times) >= self.requests_per_minute:
            # Calculate how long to wait
            oldest_request = self.request_times[0]
            wait_until = oldest_request + timedelta(minutes=1)
            wait_seconds = (wait_until - now).total_seconds()
            
            if wait_seconds > 0:
                print(f"⏳ Approaching rate limit. Waiting {wait_seconds:.1f} seconds...")
                time.sleep(wait_seconds + 0.1)  # Add small buffer
    
    def make_request(self, messages):
        """Make a request with rate limit handling"""
        self.wait_if_needed()
        
        try:
            # Record request time
            self.request_times.append(datetime.now())
            
            response = self.client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=messages
            )
            
            # Check response headers for rate limit info
            # Note: OpenAI Python client may not expose all headers
            
            return response.choices[0].message.content, None
            
        except openai.RateLimitError as e:
            self.last_rate_limit_time = datetime.now()
            
            # Parse retry-after if available
            retry_after = getattr(e, 'retry_after', None)
            if retry_after:
                print(f"⏳ Rate limited. Retry after {retry_after} seconds")
                time.sleep(retry_after)
            else:
                # Default wait
                print("⏳ Rate limited. Waiting 60 seconds...")
                time.sleep(60)
            
            # Try once more
            return self.make_request(messages)
            
        except Exception as e:
            return None, f"Error: {str(e)}"
    
    def get_status(self):
        """Get current rate limit status"""
        now = datetime.now()
        one_minute_ago = now - timedelta(minutes=1)
        recent_requests = len([t for t in self.request_times if t > one_minute_ago])
        
        return {
            'recent_requests': recent_requests,
            'limit': self.requests_per_minute,
            'available': self.requests_per_minute - recent_requests
        }

# Setup
api_key = load_api_key()
if not api_key:
    print("❌ No API key found!")
    exit()

client = openai.OpenAI(api_key=api_key)
rate_handler = RateLimitHandler(client)

print("🚦 Rate Limit Handler Demo")
print("=" * 50)
print(f"Rate limit: {rate_handler.requests_per_minute} requests/minute")
print("Commands: 'status', 'burst', 'quit'")
print("-" * 50)

while True:
    command = input("\nEnter command or message: ").strip()
    
    if command.lower() == 'quit':
        break
    
    elif command.lower() == 'status':
        status = rate_handler.get_status()
        print(f"\n📊 Rate Limit Status:")
        print(f"  Recent requests: {status['recent_requests']}")
        print(f"  Limit: {status['limit']}/minute")
        print(f"  Available: {status['available']}")
        continue
    
    elif command.lower() == 'burst':
        # Test rate limiting with burst requests
        print("\n🚀 Sending burst of requests...")
        for i in range(5):
            print(f"\nRequest {i+1}/5:")
            messages = [{"role": "user", "content": f"Say '{i+1}'"}]
            response, error = rate_handler.make_request(messages)
            
            if response:
                print(f"✅ Response: {response}")
            else:
                print(f"❌ Error: {error}")
            
            # Show status
            status = rate_handler.get_status()
            print(f"   [{status['recent_requests']}/{status['limit']} requests used]")
        continue
    
    # Normal message
    messages = [{"role": "user", "content": command}]
    response, error = rate_handler.make_request(messages)
    
    if response:
        print(f"\n🤖 Response: {response}")
        status = rate_handler.get_status()
        print(f"📊 Requests: {status['recent_requests']}/{status['limit']}")
    else:
        print(f"\n❌ {error}")

print("\n👋 Goodbye!")


In [ ]:
# From: friendly_errors.py

# From: Zero to AI Agent, Chapter 8, Section 8.4
# File: friendly_errors.py

import openai
from pathlib import Path

def load_api_key():
    env_file = Path(".env")
    if env_file.exists():
        with open(env_file, 'r') as f:
            for line in f:
                if line.startswith('OPENAI_API_KEY='):
                    return line.split('=')[1].strip()
    return None

def friendly_error_message(error):
    """Convert technical errors into user-friendly messages"""
    
    error_type = type(error).__name__
    
    friendly_messages = {
        'AuthenticationError': {
            'message': "🔑 There's an issue with the API key.",
            'suggestion': "Please check that your API key is valid and properly set up.",
            'action': "Visit https://platform.openai.com/api-keys to verify your key."
        },
        'RateLimitError': {
            'message': "⏳ We're sending too many requests too quickly.",
            'suggestion': "Please wait a moment before trying again.",
            'action': "Try again in about 60 seconds, or upgrade your API plan for higher limits."
        },
        'BadRequestError': {
            'message': "❌ The request couldn't be processed.",
            'suggestion': "There might be an issue with the message format.",
            'action': "Try rephrasing your message or making it shorter."
        },
        'APIConnectionError': {
            'message': "🌐 Can't connect to the AI service.",
            'suggestion': "Please check your internet connection.",
            'action': "Make sure you're connected to the internet and try again."
        },
        'APITimeoutError': {
            'message': "⏱️ The request took too long.",
            'suggestion': "The service might be busy right now.",
            'action': "Try again in a moment, or try a simpler request."
        },
        'ServiceUnavailableError': {
            'message': "🔧 The AI service is temporarily unavailable.",
            'suggestion': "OpenAI's servers might be under maintenance.",
            'action': "Please try again in a few minutes."
        }
    }
    
    # Get friendly message or default
    if error_type in friendly_messages:
        return friendly_messages[error_type]
    else:
        return {
            'message': f"😕 An unexpected error occurred.",
            'suggestion': f"Error type: {error_type}",
            'action': "Try again, or contact support if the problem persists."
        }

def make_safe_request(client, messages):
    """Make a request with friendly error handling"""
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages
        )
        return response.choices[0].message.content, None
    
    except Exception as e:
        return None, friendly_error_message(e)

# Setup
api_key = load_api_key()
if not api_key:
    print("❌ No API key found!")
    print("💡 Please create a .env file with your OpenAI API key.")
    print("📝 Format: OPENAI_API_KEY=sk-...")
    exit()

client = openai.OpenAI(api_key=api_key)

print("😊 Friendly Error Handler")
print("=" * 50)
print("All errors are explained in a helpful way!")
print("Commands: 'test_error', 'quit'")
print("-" * 50)

while True:
    user_input = input("\nYour message: ").strip()
    
    if user_input.lower() == 'quit':
        break
    
    elif user_input.lower() == 'test_error':
        # Intentionally trigger different errors for testing
        print("\n🧪 Testing error messages...")
        
        # Test with bad model
        try:
            client.chat.completions.create(
                model="gpt-99",
                messages=[{"role": "user", "content": "Hi"}]
            )
        except Exception as e:
            error_info = friendly_error_message(e)
            print(f"\n{error_info['message']}")
            print(f"💡 {error_info['suggestion']}")
            print(f"➡️ {error_info['action']}")
        continue
    
    # Normal request
    messages = [{"role": "user", "content": user_input}]
    response, error_info = make_safe_request(client, messages)
    
    if response:
        print(f"\n🤖 {response}")
    else:
        print(f"\n{error_info['message']}")
        print(f"💡 {error_info['suggestion']}")
        print(f"➡️ {error_info['action']}")

print("\n👋 Thanks for chatting!")


---
### Section 8.4 Exercises

### Exercise 8.4.1: Error Logger

Create a tool that:
- Logs all API errors to a file
- Tracks error frequency
- Identifies patterns (like rate limits at certain times)
- Generates an error report

In [ ]:
# Your code here


### Exercise 8.4.2: Resilient Caller

Build a function that:
- Tries different models if one fails
- Falls back to simpler requests on errors
- Maintains a "health score" for the API
- Automatically adjusts behavior based on errors

In [ ]:
# Your code here


### Exercise 8.4.3: Circuit Breaker

Implement a circuit breaker pattern that:
- Stops making requests after repeated failures
- Waits before trying again
- Gradually increases request rate when healthy
- Provides status updates to the user

In [ ]:
# Your code here


---
## Section 8.5: Building a simple chatbot

In [ ]:
# From: chatbot_core.py

# From: Zero to AI Agent, Chapter 8, Section 8.5
# File: chatbot_core.py

"""Core chatbot functionality - keep it simple and reusable!"""

class ChatBot:
    """Basic chatbot that maintains conversation context"""
    
    def __init__(self, client, system_message="You are a helpful assistant."):
        self.client = client
        self.messages = [{"role": "system", "content": system_message}]
    
    def chat(self, user_message):
        """Send a message and get a response"""
        # Add user message to history
        self.messages.append({"role": "user", "content": user_message})
        
        # Get AI response
        response = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=self.messages
        )
        
        # Extract and store AI message
        ai_message = response.choices[0].message.content
        self.messages.append({"role": "assistant", "content": ai_message})
        
        return ai_message
    
    def reset(self):
        """Reset conversation, keep system message"""
        self.messages = self.messages[:1]


In [ ]:
# From: api_helper.py

# From: Zero to AI Agent, Chapter 8, Section 8.5
# File: api_helper.py

"""Helper functions for API setup"""

from pathlib import Path
import openai

def get_client():
    """Load API key and return OpenAI client"""
    env_file = Path(".env")
    if not env_file.exists():
        raise FileNotFoundError("No .env file found! Please create one with your API key.")
    
    with open(env_file, 'r') as f:
        for line in f:
            if line.startswith('OPENAI_API_KEY='):
                api_key = line.split('=')[1].strip()
                return openai.OpenAI(api_key=api_key)
    
    raise ValueError("No OPENAI_API_KEY found in .env file!")


In [ ]:
# From: simple_chat.py

# From: Zero to AI Agent, Chapter 8, Section 8.5
# File: simple_chat.py

from chatbot_core import ChatBot
from api_helper import get_client

def main():
    # Setup
    client = get_client()
    bot = ChatBot(client)
    
    print("💬 Simple Chatbot")
    print("Type 'quit' to exit, 'reset' to start over")
    print("-" * 40)
    
    while True:
        user_input = input("\nYou: ").strip()
        
        if user_input.lower() == 'quit':
            break
        elif user_input.lower() == 'reset':
            bot.reset()
            print("🔄 Conversation reset!")
            continue
        
        response = bot.chat(user_input)
        print(f"Bot: {response}")

if __name__ == "__main__":
    main()


In [ ]:
# From: personalities.py

# From: Zero to AI Agent, Chapter 8, Section 8.5
# File: personalities.py

"""Chatbot personality definitions"""

PERSONALITIES = {
    "friendly": "You are a warm, friendly assistant. Use casual language and be encouraging!",
    
    "professional": "You are a formal, professional assistant. Be concise and business-like.",
    
    "creative": "You are a creative, imaginative assistant. Be playful and think outside the box!",
    
    "teacher": "You are a patient teacher. Explain things clearly and check understanding.",
    
    "pirate": "Ahoy! You're a pirate assistant. Talk like a pirate and use nautical terms!"
}

def get_personality(name):
    """Get a personality prompt by name"""
    return PERSONALITIES.get(name, PERSONALITIES["friendly"])

def list_personalities():
    """Get list of available personalities"""
    return list(PERSONALITIES.keys())


In [ ]:
# From: personality_chat.py

# From: Zero to AI Agent, Chapter 8, Section 8.5
# File: personality_chat.py

from chatbot_core import ChatBot
from api_helper import get_client
from personalities import get_personality, list_personalities

def main():
    client = get_client()
    
    # Show available personalities
    print("🎭 Choose a personality:")
    for p in list_personalities():
        print(f"  - {p}")
    
    choice = input("\nYour choice: ").strip().lower()
    personality = get_personality(choice)
    
    # Create bot with chosen personality
    bot = ChatBot(client, system_message=personality)
    
    print(f"\n💬 Chatbot with {choice} personality")
    print("Type 'quit' to exit")
    print("-" * 40)
    
    while True:
        user_input = input("\nYou: ").strip()
        
        if user_input.lower() == 'quit':
            break
        
        response = bot.chat(user_input)
        print(f"Bot: {response}")

if __name__ == "__main__":
    main()


In [ ]:
# From: memory_manager.py

# From: Zero to AI Agent, Chapter 8, Section 8.5
# File: memory_manager.py

"""Manage conversation memory intelligently"""

class MemoryManager:
    """Manage conversation context window"""
    
    def __init__(self, max_messages=20):
        self.max_messages = max_messages
    
    def get_context(self, messages):
        """Get messages that fit in context window"""
        if len(messages) <= self.max_messages:
            return messages
        
        # Always keep system message + recent messages
        return [messages[0]] + messages[-(self.max_messages-1):]
    
    def should_truncate(self, messages):
        """Check if truncation is needed"""
        return len(messages) > self.max_messages


In [ ]:
# From: smart_chatbot.py

# From: Zero to AI Agent, Chapter 8, Section 8.5
# File: smart_chatbot.py

"""Smarter chatbot with memory management"""

from memory_manager import MemoryManager

class SmartChatBot:
    """Chatbot with intelligent memory management"""
    
    def __init__(self, client, system_message="You are a helpful assistant.", max_context=20):
        self.client = client
        self.system_message = {"role": "system", "content": system_message}
        self.messages = [self.system_message]
        self.memory = MemoryManager(max_context)
    
    def chat(self, user_message):
        """Chat with smart context management"""
        # Add user message
        self.messages.append({"role": "user", "content": user_message})
        
        # Get context window for API call
        context = self.memory.get_context(self.messages)
        
        # Make API call with managed context
        response = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context
        )
        
        # Store response
        ai_message = response.choices[0].message.content
        self.messages.append({"role": "assistant", "content": ai_message})
        
        return ai_message
    
    def get_stats(self):
        """Get conversation statistics"""
        return {
            "total_messages": len(self.messages) - 1,  # Exclude system
            "context_size": len(self.memory.get_context(self.messages)),
            "truncated": self.memory.should_truncate(self.messages)
        }


In [ ]:
# From: commands.py

# From: Zero to AI Agent, Chapter 8, Section 8.5
# File: commands.py

"""Command handling for chatbots"""

class CommandHandler:
    """Handle special commands in chat"""
    
    def __init__(self):
        self.commands = {
            '/help': self.show_help,
            '/stats': self.show_stats,
            '/clear': self.clear_chat
        }
        self.bot = None  # Set by chatbot
    
    def set_bot(self, bot):
        """Connect to a chatbot instance"""
        self.bot = bot
    
    def handle(self, input_text):
        """Check if input is a command and handle it"""
        if not input_text.startswith('/'):
            return None
        
        command = input_text.split()[0]
        return self.commands.get(command, self.unknown_command)()
    
    def show_help(self):
        """Show available commands"""
        return """Available commands:
  /help  - Show this help
  /stats - Show conversation stats
  /clear - Clear conversation"""
    
    def show_stats(self):
        """Show conversation statistics"""
        if self.bot and hasattr(self.bot, 'get_stats'):
            stats = self.bot.get_stats()
            return f"Messages: {stats['total_messages']}, Context: {stats['context_size']}"
        return "Stats not available"
    
    def clear_chat(self):
        """Clear conversation"""
        if self.bot and hasattr(self.bot, 'messages'):
            self.bot.messages = self.bot.messages[:1]
            return "Conversation cleared!"
        return "Cannot clear"
    
    def unknown_command(self):
        """Handle unknown commands"""
        return "Unknown command. Type /help for available commands."


In [ ]:
# From: complete_chat.py

# From: Zero to AI Agent, Chapter 8, Section 8.5
# File: complete_chat.py

from smart_chatbot import SmartChatBot
from commands import CommandHandler
from personalities import get_personality
from api_helper import get_client

def main():
    # Setup
    client = get_client()
    bot = SmartChatBot(client, max_context=10)
    
    # Add command handling
    cmd_handler = CommandHandler()
    cmd_handler.set_bot(bot)
    
    print("🤖 Complete Modular Chatbot")
    print("Type /help for commands, 'quit' to exit")
    print("-" * 40)
    
    while True:
        user_input = input("\nYou: ").strip()
        
        if user_input.lower() == 'quit':
            break
        
        # Check for commands
        cmd_response = cmd_handler.handle(user_input)
        if cmd_response:
            print(f"\n{cmd_response}")
            continue
        
        # Regular chat
        response = bot.chat(user_input)
        print(f"Bot: {response}")
        
        # Show truncation warning if needed
        stats = bot.get_stats()
        if stats['truncated']:
            print(f"[Context limited to last {stats['context_size']} messages]")

if __name__ == "__main__":
    main()


In [ ]:
# From: demo_modular.py

# From: Zero to AI Agent, Chapter 8, Section 8.5
# File: demo_modular.py

"""Demonstrate the modular architecture"""

from chatbot_core import ChatBot
from api_helper import get_client
from personalities import get_personality
from memory_manager import MemoryManager

def demonstrate_modules():
    """Show how modules work together"""
    
    print("🧩 Modular Chatbot Architecture Demo")
    print("=" * 50)
    
    # 1. API Setup (one line!)
    client = get_client()
    print("✅ API client ready")
    
    # 2. Choose personality (one line!)
    personality = get_personality("friendly")
    print("✅ Personality loaded")
    
    # 3. Create chatbot (one line!)
    bot = ChatBot(client, personality)
    print("✅ Chatbot created")
    
    # 4. Memory manager (one line!)
    memory = MemoryManager(max_messages=10)
    print("✅ Memory manager ready")
    
    print("\n" + "=" * 50)
    print("Each module is independent and reusable!")
    print("Mix and match them however you need!")
    
    # Example: Quick test
    response = bot.chat("Hello! What's 2+2?")
    print(f"\nTest chat: {response}")
    
    # Show context management
    test_messages = [{"role": "user", "content": f"Message {i}"} for i in range(15)]
    context = memory.get_context(test_messages)
    print(f"\n15 messages → {len(context)} in context (truncated: {memory.should_truncate(test_messages)})")

if __name__ == "__main__":
    demonstrate_modules()


In [ ]:
# From: response_filters.py

# From: Zero to AI Agent, Chapter 8, Section 8.5
# File: response_filters.py

"""Filter and modify chatbot responses"""

class ResponseFilter:
    """Modify responses before showing to user"""
    
    @staticmethod
    def make_brief(response, max_length=100):
        """Shorten long responses"""
        if len(response) <= max_length:
            return response
        return response[:max_length-3] + "..."
    
    @staticmethod
    def add_emoji(response):
        """Add relevant emoji to responses"""
        emoji_map = {
            'happy': '😊', 'sad': '😔', 'hello': '👋',
            'yes': '✅', 'no': '❌', 'think': '🤔',
            'love': '❤️', 'great': '🎉', 'sorry': '😅'
        }
        
        response_lower = response.lower()
        for word, emoji in emoji_map.items():
            if word in response_lower:
                return f"{emoji} {response}"
        return response
    
    @staticmethod
    def make_uppercase(response):
        """MAKE RESPONSE LOUDER!"""
        return response.upper()


In [ ]:
# From: filtered_chat.py

# From: Zero to AI Agent, Chapter 8, Section 8.5
# File: filtered_chat.py

from chatbot_core import ChatBot
from api_helper import get_client
from response_filters import ResponseFilter

def main():
    client = get_client()
    bot = ChatBot(client)
    filter = ResponseFilter()
    
    print("💬 Chatbot with Response Filters")
    print("Commands: 'brief', 'emoji', 'loud', 'normal'")
    print("-" * 40)
    
    filter_mode = "normal"
    
    while True:
        user_input = input("\nYou: ").strip()
        
        if user_input.lower() == 'quit':
            break
        elif user_input.lower() in ['brief', 'emoji', 'loud', 'normal']:
            filter_mode = user_input.lower()
            print(f"Filter set to: {filter_mode}")
            continue
        
        # Get response
        response = bot.chat(user_input)
        
        # Apply filter
        if filter_mode == 'brief':
            response = filter.make_brief(response)
        elif filter_mode == 'emoji':
            response = filter.add_emoji(response)
        elif filter_mode == 'loud':
            response = filter.make_uppercase(response)
        
        print(f"Bot: {response}")

if __name__ == "__main__":
    main()


---
### Section 8.5 Exercises

### Exercise 8.5.1: Topic Tracker

Create a module that:
- Tracks what topics have been discussed
- Counts how often each topic appears
- Can report on conversation themes

In [ ]:
# Your code here


### Exercise 8.5.2: Response Timer

Create a module that:
- Times how long API calls take
- Tracks average response time
- Warns if responses are slow

In [ ]:
# Your code here


### Exercise 8.5.3: Mood Detector

Create a module that:
- Analyzes user message sentiment
- Adjusts bot personality based on mood
- Tracks mood over the conversation

In [ ]:
# Your code here


---
## Section 8.6: Saving conversation history

In [ ]:
# From: simple_saver.py

# From: Zero to AI Agent, Chapter 8, Section 8.6
# File: simple_saver.py

import json
from datetime import datetime
from pathlib import Path

def save_conversation(messages, filename=None):
    """Save a conversation to a JSON file"""
    # Auto-generate filename if not provided
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"conversation_{timestamp}.json"
    
    # Create the data structure
    conversation_data = {
        "saved_at": datetime.now().isoformat(),
        "message_count": len(messages),
        "messages": messages
    }
    
    # Save to file
    with open(filename, 'w') as f:
        json.dump(conversation_data, f, indent=2)
    
    print(f"💾 Conversation saved to {filename}")
    return filename

# Example usage
sample_conversation = [
    {"role": "user", "content": "What is Python?"},
    {"role": "assistant", "content": "Python is a high-level programming language..."},
    {"role": "user", "content": "Show me an example"},
    {"role": "assistant", "content": "Here's a simple example: print('Hello, World!')"}
]

# Save it!
save_conversation(sample_conversation)


In [ ]:
# From: simple_loader.py

# From: Zero to AI Agent, Chapter 8, Section 8.6
# File: simple_loader.py

import json
from pathlib import Path

def load_conversation(filename):
    """Load a conversation from a JSON file"""
    file_path = Path(filename)
    
    # Check if file exists
    if not file_path.exists():
        print(f"❌ File {filename} not found!")
        return None
    
    # Load the data
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    print(f"✅ Loaded {data['message_count']} messages")
    print(f"📅 Saved on: {data['saved_at']}")
    
    return data['messages']

def list_saved_conversations():
    """Find all saved conversation files"""
    # Look for conversation files
    files = list(Path(".").glob("conversation_*.json"))
    
    if not files:
        print("No saved conversations found!")
        return []
    
    print(f"\n📚 Found {len(files)} saved conversations:")
    for i, file in enumerate(files, 1):
        # Get file info
        size = file.stat().st_size / 1024  # Size in KB
        modified = datetime.fromtimestamp(file.stat().st_mtime)
        
        print(f"  {i}. {file.name}")
        print(f"     Size: {size:.1f} KB")
        print(f"     Modified: {modified.strftime('%Y-%m-%d %H:%M')}")
    
    return files

# Try it out
files = list_saved_conversations()
if files:
    # Load the first one
    messages = load_conversation(files[0])


In [ ]:
# From: readable_format.py

# From: Zero to AI Agent, Chapter 8, Section 8.6
# File: readable_format.py

def save_as_text(messages, filename=None):
    """Save conversation in human-readable text format"""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"chat_transcript_{timestamp}.txt"
    
    with open(filename, 'w') as f:
        # Write header
        f.write("=" * 60 + "\n")
        f.write(f"CONVERSATION TRANSCRIPT\n")
        f.write(f"Date: {datetime.now().strftime('%B %d, %Y at %I:%M %p')}\n")
        f.write("=" * 60 + "\n\n")
        
        # Write each message
        for i, msg in enumerate(messages, 1):
            role = msg['role'].upper()
            content = msg['content']
            
            # Format based on role
            if role == 'USER':
                f.write(f"👤 YOU (Message {i}):\n")
            elif role == 'ASSISTANT':
                f.write(f"🤖 AI (Message {i}):\n")
            else:
                f.write(f"📋 {role} (Message {i}):\n")
            
            f.write(f"{content}\n")
            f.write("-" * 40 + "\n\n")
        
        # Write footer
        f.write("=" * 60 + "\n")
        f.write(f"End of conversation - {len(messages)} messages total\n")
    
    print(f"📄 Readable transcript saved to {filename}")
    return filename


In [ ]:
# From: conversation_manager.py

# From: Zero to AI Agent, Chapter 8, Section 8.6
# File: conversation_manager.py

import json
from pathlib import Path
from datetime import datetime

class ConversationManager:
    """Manage conversation history like a pro"""
    
    def __init__(self, storage_dir="my_conversations"):
        """Set up our conversation storage system"""
        # Create a dedicated directory for conversations
        self.storage_dir = Path(storage_dir)
        self.storage_dir.mkdir(exist_ok=True)
        
        # Track current conversation
        self.current_conversation = []
        self.conversation_metadata = {}
        
        print(f"📁 Conversation storage initialized in '{storage_dir}/'")
    
    def start_new_conversation(self, title=None):
        """Start a fresh conversation"""
        # Save previous conversation if it exists
        if self.current_conversation:
            self.save_current_conversation()
        
        # Reset for new conversation
        self.current_conversation = []
        self.conversation_metadata = {
            'id': datetime.now().strftime('%Y%m%d_%H%M%S'),
            'title': title or f"Chat {datetime.now().strftime('%I:%M %p')}",
            'started': datetime.now().isoformat()
        }
        
        print(f"💬 Started new conversation: {self.conversation_metadata['title']}")
        return self.conversation_metadata['id']
    
    def add_message(self, role, content):
        """Add a message to the current conversation"""
        message = {
            'role': role,
            'content': content,
            'timestamp': datetime.now().isoformat()
        }
        self.current_conversation.append(message)
        
        # Auto-save every 10 messages
        if len(self.current_conversation) % 10 == 0:
            self.save_current_conversation()
            print("💾 Auto-saved (10 messages reached)")
    
    def save_current_conversation(self):
        """Save the current conversation"""
        if not self.current_conversation:
            return None
        
        # Update metadata
        self.conversation_metadata['ended'] = datetime.now().isoformat()
        self.conversation_metadata['message_count'] = len(self.current_conversation)
        
        # Create filename using ID
        conv_id = self.conversation_metadata['id']
        filename = self.storage_dir / f"conversation_{conv_id}.json"
        
        # Save everything
        data = {
            'metadata': self.conversation_metadata,
            'messages': self.current_conversation
        }
        
        print(f"dumping to {filename}")

        with open(filename, 'w') as f:
            json.dump(data, f, indent=2)
        
        print(f"💾 Saved: {self.conversation_metadata['title']}")
        return filename


In [ ]:
# From: searchable_history.py

# From: Zero to AI Agent, Chapter 8, Section 8.6
# File: searchable_history.py

import json
from pathlib import Path

def search_conversations(search_term, storage_dir="my_conversations"):
    """Search through all saved conversations"""
    storage_path = Path(storage_dir)
    results = []
    search_lower = search_term.lower()
    
    print(f"🔍 Searching for '{search_term}'...")
    
    # Check if directory exists
    if not storage_path.exists():
        print(f"❌ Directory {storage_dir} doesn't exist!")
        return []
    
    # Search through each conversation file
    for conv_file in storage_path.glob("conversation_*.json"):
        try:
            with open(conv_file, 'r') as f:
                data = json.load(f)
            
            # Search in messages
            for msg in data.get('messages', []):
                if search_lower in msg.get('content', '').lower():
                    # Found a match!
                    metadata = data.get('metadata', {})
                    results.append({
                        'file': conv_file.name,
                        'title': metadata.get('title', 'Untitled'),
                        'date': metadata.get('started', data.get('saved_at', 'Unknown')),
                        'preview': msg['content'][:100] + "..." if len(msg['content']) > 100 else msg['content'],
                        'role': msg['role'],
                        'full_path': str(conv_file)
                    })
                    break  # One result per conversation
        except Exception as e:
            print(f"⚠️ Error reading {conv_file.name}: {e}")
            continue
    
    # Display results
    if results:
        print(f"\n✅ Found {len(results)} conversations containing '{search_term}':\n")
        for i, result in enumerate(results, 1):
            print(f"{i}. {result['title']}")
            print(f"   Date: {result['date'][:19] if len(result['date']) > 19 else result['date']}")
            print(f"   Preview: {result['preview']}")
            print(f"   File: {result['file']}")
            print()
    else:
        print(f"❌ No conversations found containing '{search_term}'")
    
    return results

def search_advanced(storage_dir="my_conversations", **criteria):
    """Advanced search with multiple criteria"""
    storage_path = Path(storage_dir)
    results = []
    
    # Search criteria
    search_term = criteria.get('term', '').lower()
    role_filter = criteria.get('role', None)  # 'user' or 'assistant'
    date_from = criteria.get('date_from', None)
    date_to = criteria.get('date_to', None)
    min_messages = criteria.get('min_messages', 0)
    
    if not storage_path.exists():
        return []
    
    for conv_file in storage_path.glob("conversation_*.json"):
        try:
            with open(conv_file, 'r') as f:
                data = json.load(f)
            
            # Check message count
            if len(data.get('messages', [])) < min_messages:
                continue
            
            # Check date range if specified
            if date_from or date_to:
                conv_date = data.get('saved_at', data.get('metadata', {}).get('started', ''))
                if date_from and conv_date < date_from:
                    continue
                if date_to and conv_date > date_to:
                    continue
            
            # Search in messages
            for msg in data.get('messages', []):
                # Check role filter
                if role_filter and msg.get('role') != role_filter:
                    continue
                
                # Check search term
                if search_term and search_term not in msg.get('content', '').lower():
                    continue
                
                # Match found!
                metadata = data.get('metadata', {})
                results.append({
                    'file': conv_file.name,
                    'title': metadata.get('title', 'Untitled'),
                    'date': metadata.get('started', data.get('saved_at', 'Unknown')),
                    'preview': msg['content'][:100] + "..." if len(msg['content']) > 100 else msg['content'],
                    'role': msg['role'],
                    'message_count': len(data.get('messages', [])),
                    'full_path': str(conv_file)
                })
                break
                
        except Exception as e:
            continue
    
    return results

def load_conversation_for_viewing(filepath):
    """Load a specific conversation for viewing"""
    try:
        with open(filepath, 'r') as f:
            data = json.load(f)
        
        print(f"\n📂 Loading: {Path(filepath).name}")
        print("=" * 50)
        
        # Show metadata
        metadata = data.get('metadata', {})
        print(f"Title: {metadata.get('title', 'Untitled')}")
        print(f"Started: {metadata.get('started', 'Unknown')}")
        print(f"Messages: {len(data.get('messages', []))}")
        print("-" * 50)
        
        # Show messages
        for msg in data.get('messages', []):
            role = "YOU" if msg['role'] == 'user' else "AI"
            print(f"\n{role}:")
            print(msg['content'])
            print("-" * 40)
        
        return data
        
    except Exception as e:
        print(f"❌ Error loading conversation: {e}")
        return None

# Example usage
if __name__ == "__main__":
    # Simple search
    print("🔍 Simple Search Demo")
    print("=" * 50)
    results = search_conversations("python")
    
    # Advanced search
    print("\n🔍 Advanced Search Demo")
    print("=" * 50)
    advanced_results = search_advanced(
        term="code",
        role="assistant",
        min_messages=5
    )
    
    if advanced_results:
        print(f"Found {len(advanced_results)} conversations with advanced criteria")
        for result in advanced_results[:3]:  # Show first 3
            print(f"- {result['title']} ({result['message_count']} messages)")
    
    # Load and view a specific conversation
    if results:
        print("\n📖 Loading first search result...")
        load_conversation_for_viewing(results[0]['full_path'])

In [ ]:
# From: auto_summary.py

# From: Zero to AI Agent, Chapter 8, Section 8.6
# File: auto_summary.py

import openai

def create_conversation_summary(messages, client):
    """Generate a summary of a conversation using AI"""
    
    # Don't summarize very short conversations
    if len(messages) < 5:
        return "Conversation too short to summarize"
    
    # Prepare the conversation as text
    conversation_text = ""
    for msg in messages[:20]:  # Limit to prevent token overflow
        role = "Human" if msg['role'] == 'user' else "AI"
        conversation_text += f"{role}: {msg['content']}\n\n"
    
    # Ask AI to summarize
    prompt = f"""Please summarize this conversation in 2-3 sentences. 
Focus on the main topics discussed and any conclusions reached:

{conversation_text}

Summary:"""
    
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.5,  # Lower temperature for factual summary
            max_tokens=100
        )
        
        summary = response.choices[0].message.content
        return summary
    
    except Exception as e:
        return f"Could not generate summary: {str(e)}"

def save_with_summary(messages, client, filename=None):
    """Save conversation with an auto-generated summary"""
    
    # Generate summary
    print("📝 Generating summary...")
    summary = create_conversation_summary(messages, client)
    
    # Prepare data
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"summarized_chat_{timestamp}.json"
    
    data = {
        "saved_at": datetime.now().isoformat(),
        "summary": summary,
        "message_count": len(messages),
        "messages": messages
    }
    
    # Save
    with open(filename, 'w') as f:
        json.dump(data, f, indent=2)
    
    print(f"💾 Saved with summary: {summary}")
    return filename


In [ ]:
# From: complete_history_system.py

# From: Zero to AI Agent, Chapter 8, Section 8.6
# File: complete_history_system.py

import openai
from pathlib import Path
from conversation_manager import ConversationManager
from searchable_history import search_conversations
from auto_summary import save_with_summary

def load_api_key():
    env_file = Path(".env")
    if env_file.exists():
        with open(env_file, 'r') as f:
            for line in f:
                if line.startswith('OPENAI_API_KEY='):
                    return line.split('=')[1].strip()
    return None

# Set up
api_key = load_api_key()
if not api_key:
    print("❌ No API key found!")
    exit()

client = openai.OpenAI(api_key=api_key)
manager = ConversationManager()

print("💾 Complete History System")
print("=" * 50)
print("Your conversations are automatically saved and searchable!")
print("\nCommands:")
print("  'new' - Start a new conversation")
print("  'search <term>' - Search conversation history")  
print("  'list' - Show recent conversations")
print("  'quit' - Exit and save")
print("-" * 50)

# Start first conversation
manager.start_new_conversation("Demo Chat")

while True:
    user_input = input("\nYou: ").strip()
    
    if user_input.lower() == 'quit':
        # Save before exiting
        manager.save_current_conversation()
        print("👋 All conversations saved. Goodbye!")
        break
    
    elif user_input.lower() == 'new':
        title = input("Conversation title (or Enter for default): ").strip()
        manager.start_new_conversation(title)
        continue
    
    elif user_input.lower().startswith('search '):
        search_term = user_input[7:]
        search_conversations(search_term, manager.storage_dir)
        continue
    
    elif user_input.lower() == 'list':
        files = list(manager.storage_dir.glob("conversation_*.json"))
        print(f"\n📚 You have {len(files)} saved conversations")
        for file in files[-5:]:  # Show last 5
            print(f"  • {file.name}")
        continue
    
    # Regular conversation
    manager.add_message("user", user_input)
    
    # Get AI response (simplified for demo)
    messages = [{"role": "system", "content": "You are a helpful assistant."}]
    messages.extend(manager.current_conversation[-10:])  # Last 10 messages
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages
    )
    
    ai_message = response.choices[0].message.content
    print(f"\nAI: {ai_message}")
    
    manager.add_message("assistant", ai_message)

print("\n✨ Thank you for using the Complete History System!")


---
### Section 8.6 Exercises

### Exercise 8.6.1: Export Master

Create a tool that can export conversations to:
- HTML format with nice formatting
- CSV for spreadsheet analysis
- Markdown for documentation
- PDF for sharing (bonus challenge!)

In [ ]:
# Your code here


### Exercise 8.6.2: Smart Organizer

Build a system that:
- Automatically organizes conversations by topic
- Groups related conversations together
- Creates daily/weekly summaries
- Suggests titles based on content

In [ ]:
# Your code here


### Exercise 8.6.3: History Analytics

Create an analyzer that shows:
- Your most common topics
- Average conversation length
- Most active times of day
- Conversation trends over time

In [ ]:
# Your code here


---
## Next Steps

- Check your answers in **chapter_08_first_llm_solutions.ipynb**
- Proceed to **Chapter 9**